# Introduction
A key feature of PorePy is that forward-mode algorithmic (or automatic) differentiation, AD, is used to linearize a system of partial differential equations. This tutorial provides a short introduction to algorithmic differentiation and an overview of the AD-module included in PorePy. 

Further description is provided in other tutorials: *pdes with ad*  explains how to use the module to solve (mixed-dimensional) PDEs. For an example where the AD module has been used to solve non-linear compressible flow, see the tutorial:  "compressible_flow_with_automatic_differentiation"


In [4]:
import numpy as np
import scipy.sparse as sps

from porepy.numerics.ad.forward_mode import Ad_array
import porepy.numerics.ad.functions as af
import porepy as pp

AttributeError: partially initialized module 'porepy' has no attribute 'Grid' (most likely due to a circular import)

# AD module
Algorithmic differentiation is used to compute the derivative of a function. This is achieved by augmenting variables with a field representing the derivative in addition to the value of the variable. Evaluation of a function of such variables is performed by (repeatedly) applying the chain rule while keeping track of both values and derivatives.

## Scalar variables

We initiate a variable $x = 2$ by giving a pair (val, jac) to the Ad_array class. val is the value at which the function will be evaluated and jac =1 since $\frac{d x}{dx} = 1$.

In [ ]:
x = Ad_array(2, 1)

We can now define a function $y=x^2 + 3$ 

In [ ]:
y = x**2 + 3

To obtain the function value and the derivative we can call .val and .jac

In [ ]:
print('y value is: ', y.val)
print('dy/dx is: ', y.jac)

$y$ is also an AD variable as a function of $x$. We can use it to declare further functions, e.g., $h(x) = e^{y(x)}$. To take the exponential of an Ad_array we need to call the exponential function found in the AD module

In [ ]:
h = af.exp(y)
print('h value is: ', h.val)
print('dh/dx is: ', h.jac)

If we know the value and Jacobian of $y$ we could alternatively skip initiating $x$ and initiate $y$ directly:

In [ ]:
y = Ad_array(7, 4)
h = af.exp(y)
print('h value is: ', h.val)
print('dh/dx is: ', h.jac)

## Vector variables
The Ad_array class also support arrays.

In [ ]:
x = Ad_array(np.array([3,2,1]), sps.diags([1,1,1]))

As for the scalar case, it is straightforward to define functions using normal Python programming. Let us declare the function
$$y(x) = \mathbf{A} x + x^2 - b$$
which has the Jacobian
$$ J_y(x) = \mathbf{A} + 2 \, \mathbf{I}\, x$$
With this notation we mean $x^2 = [x_1^2, x_2^2, x_3^2]$.

In [ ]:
A = sps.csc_matrix(np.array([[3,2,1],
                             [2,6,1],
                             [2,3,4]]))
b =  Ad_array(np.array([1,0,0]), sps.diags([0,0,0]))
y = A*x  + x**2 - b

print('Analytic y value: ')
print(np.array([22, 23, 17]))
print('Analytic y Jacobian:')
print(np.array([[9,2,1],[2,10,1],[2,3,6]]),'\n')
print('Ad y value: ')
print(y.val)
print('Ad y Jacobian:')
print(y.jac.A)


## Equations
An equation is defined by expressing it in residual form
$$y(x) = 0.$$
We can find its roots (solution) using the Newton-Rhapson scheme, which, on linearized form, reads 
$$J_y(x^{k}) (x^{k+1} - x^{k}) = - y(x^{k}).$$

In [ ]:
tol = 1e-5
residuals = list()
for i in range(5): 
    x = x - sps.linalg.spsolve(y.jac, y.val)
    y = A*x  + x**2 - b
    residuals.append(np.linalg.norm(y.val))

print(f"After five iterations, the solution is {x.val}. The residuals were ")
print(residuals)


Conceptually, the example illustrates how equations can be solved in PorePy:
1. Define ad variables and parameters matrices
2. Express the equation in residual form
3. Solve the nonlinear problem using Newton-Rhapson

PDEs in mixed-dimensional form are covered in the tutorial [equation definition tutorial](https://github.com/pmgbergen/porepy/blob/develop/tutorials/equation_definition.ipynb).